# RAG Chatbot System - Interactive Demo

This notebook demonstrates the RAG (Retrieval-Augmented Generation) chatbot system with FAISS vector search, Langchain workflows, and Azure OpenAI function calling.

## Features Demonstrated:
- **Vector Search**: FAISS-based document retrieval
- **RAG Pipeline**: Langchain retrieval-augmented generation
- **Function Calling**: Dynamic external data access
- **Multiple Use Cases**: IT Helpdesk, Customer Support, HR Assistant

## 1. Setup and Imports

In [1]:
import os
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

print(f"Project root: {project_root}")

# Import our RAG system components
from rag_system.chat_interface import RAGChatbot
from rag_system.vector_store import VectorStore
from rag_system.function_calling import FunctionCaller
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
print("✅ Imports completed successfully!")

Project root: c:\Users\tuanph16\Downloads\chatbot 1\ws4


ModuleNotFoundError: No module named 'langchain_openai'

## 2. Environment Check

In [ ]:
# Check if environment variables are configured
required_vars = [
    "AZURE_OPENAI_ENDPOINT",
    "AZURE_OPENAI_API_KEY",
    "AZURE_OPENAI_API_VERSION"
]

print("Environment Configuration Check:")
print("=" * 40)

all_configured = True
for var in required_vars:
    value = os.getenv(var)
    if value:
        print(f"✅ {var}: Configured")
    else:
        print(f"❌ {var}: Missing")
        all_configured = False

if all_configured:
    print("\n🎉 All environment variables are configured!")
else:
    print("\n⚠️ Please configure missing environment variables in your .env file")
    print("Copy .env.template to .env and fill in your Azure OpenAI credentials")

## 3. Vector Store Demo

First, let's demonstrate the FAISS vector store functionality:

In [ ]:
print("🔍 Testing Vector Store...\n")

# Create vector store for IT helpdesk
vector_store = VectorStore("it_helpdesk")

# Load IT helpdesk documents
from mock_data.it_helpdesk import get_it_helpdesk_data
documents = get_it_helpdesk_data()

print(f"📚 Loaded {len(documents)} documents")
print(f"Sample document: {documents[0]['page_content'][:100]}...")

# Create or load index
vector_store.create_index(documents)

# Get stats
stats = vector_store.get_stats()
print(f"\n📊 Vector Store Stats: {stats}")

In [ ]:
# Test similarity search
test_queries = [
    "computer is slow",
    "password reset",
    "printer not working",
    "VPN connection"
]

print("🔍 Testing Similarity Search:\n")

for query in test_queries:
    print(f"Query: '{query}'")
    results = vector_store.search(query, k=2)

    for i, result in enumerate(results, 1):
        score = result['score']
        content = result['content'][:80] + "..."
        category = result['metadata'].get('category', 'N/A')
        print(f"  {i}. [{category}] Score: {score:.3f} - {content}")

    print()

## 4. Function Calling Demo

Now let's test the Azure OpenAI function calling capabilities:

In [ ]:
print("🔧 Testing Function Calling...\n")

# Create function caller for IT helpdesk
function_caller = FunctionCaller("it_helpdesk")

# Show available functions
functions = list(function_caller.functions.keys())
print(f"📋 Available Functions ({len(functions)}):")
for func in functions:
    description = function_caller.functions[func].description
    print(f"  • {func}: {description}")

print()

In [ ]:
# Test individual functions
print("🧪 Testing Individual Functions:\n")

# Test device status check
device_result = function_caller.call_function(
    "check_device_status",
    {"device_id": "printer01"}
)
print(f"Device Status for 'printer01': {device_result}")

# Test software info
software_result = function_caller.call_function(
    "get_software_info",
    {"software_name": "microsoft_office"}
)
print(f"\nSoftware Info for 'microsoft_office': {software_result}")

# Test solution search
solution_result = function_caller.call_function(
    "search_it_solutions",
    {"keywords": ["slow", "performance"]}
)
print(f"\nSolutions for 'slow performance': {solution_result}")

## 5. Complete RAG Chatbot Demo

Now let's test the complete RAG chatbot with both retrieval and function calling:

In [ ]:
print("🤖 Initializing Complete RAG Chatbot...\n")

# Create chatbot for IT helpdesk
chatbot = RAGChatbot(use_case="it_helpdesk", enable_functions=True)

# Get chatbot statistics
stats = chatbot.get_chatbot_stats()
print("📊 Chatbot Statistics:")
print(f"  Use Case: {stats['use_case']}")
print(f"  Functions Enabled: {stats['functions_enabled']}")
print(f"  Available Functions: {stats['functions']['total_functions']}")
print(f"  Vector Store Documents: {stats['retrieval_chain']['vector_store']['total_documents']}")
print()

### Test Different Types of Queries

In [ ]:
# Test queries that should use RAG retrieval
rag_queries = [
    "My computer is running very slowly, what should I do?",
    "How do I reset my password?",
    "I can't connect to the company VPN"
]

print("📚 Testing RAG Retrieval Queries:\n")

for i, query in enumerate(rag_queries, 1):
    print(f"{i}. Query: {query}")
    print("-" * 50)

    response = chatbot.chat(query)

    print(f"Answer: {response['answer']}")
    print(f"Method: {response['method']}")

    if response.get('sources'):
        print(f"Sources: {', '.join(response['sources'])}")

    print("\n" + "=" * 80 + "\n")

In [ ]:
# Test queries that should trigger function calls
function_queries = [
    "What's the status of printer01?",
    "Can you check if server02 is working?",
    "Tell me about Visual Studio software"
]

print("🔧 Testing Function Calling Queries:\n")

for i, query in enumerate(function_queries, 1):
    print(f"{i}. Query: {query}")
    print("-" * 50)

    response = chatbot.chat(query)

    print(f"Answer: {response['answer']}")
    print(f"Method: {response['method']}")

    if response.get('function_calls_made'):
        print(f"Function Calls Made: {response['function_calls_made']}")

    print("\n" + "=" * 80 + "\n")

## 6. Multi-Use Case Demo

Let's test different use cases to show the versatility of the system:

In [ ]:
use_cases = ["customer_support", "hr_assistant"]

test_scenarios = {
    "customer_support": [
        "How can I track my order?",
        "What's the status of order ORD123456?",
        "How much would shipping cost for a $75 order?"
    ],
    "hr_assistant": [
        "How do I request time off?",
        "What's my leave balance for employee EMP001?",
        "What are the company holidays for 2025?"
    ]
}

for use_case in use_cases:
    print(f"🔄 Testing {use_case.title()} Use Case\n")

    # Create chatbot for this use case
    case_chatbot = RAGChatbot(use_case=use_case, enable_functions=True)

    queries = test_scenarios[use_case]

    for query in queries:
        print(f"Q: {query}")
        response = case_chatbot.chat(query)

        # Truncate long responses for readability
        answer = response['answer']
        if len(answer) > 200:
            answer = answer[:200] + "..."

        print(f"A: {answer}")
        print(f"Method: {response['method']}")
        print()

    print("=" * 80 + "\n")

## 7. Interactive Chat Session

Create an interactive chat session (run this cell and type your questions):

In [ ]:
def interactive_chat_session():
    """Run an interactive chat session in the notebook."""

    print("🤖 Interactive Chat Session Started!")
    print("Choose your use case: 1) IT Helpdesk, 2) Customer Support, 3) HR Assistant")
    print("Type 'quit' to exit, 'switch' to change use case\n")

    # Default chatbot
    current_chatbot = RAGChatbot("it_helpdesk")
    current_use_case = "it_helpdesk"

    print(f"Current use case: {current_use_case}")
    print("Type your question below:")

    while True:
        try:
            user_input = input("\nYou: ").strip()

            if user_input.lower() in ['quit', 'exit']:
                print("Goodbye! 👋")
                break

            elif user_input.lower() == 'switch':
                print("Available use cases:")
                print("1. it_helpdesk")
                print("2. customer_support")
                print("3. hr_assistant")

                choice = input("Enter choice (1-3): ").strip()
                use_case_map = {
                    '1': 'it_helpdesk',
                    '2': 'customer_support',
                    '3': 'hr_assistant'
                }

                if choice in use_case_map:
                    current_use_case = use_case_map[choice]
                    current_chatbot = RAGChatbot(current_use_case)
                    print(f"Switched to {current_use_case}")
                else:
                    print("Invalid choice")
                continue

            elif not user_input:
                continue

            # Process the question
            response = current_chatbot.chat(user_input)

            print(f"\n🤖 Bot: {response['answer']}")

            # Show method used
            method_icons = {
                'function_calling': '🔧',
                'rag_retrieval': '📚',
                'fallback': '💭'
            }

            icon = method_icons.get(response['method'], '🤖')
            print(f"\n{icon} Method: {response['method']}")

            if response.get('function_calls_made'):
                print(f"🔧 Function calls: {response['function_calls_made']}")

            if response.get('sources'):
                print(f"📚 Sources: {', '.join(response['sources'][:2])}")

        except KeyboardInterrupt:
            print("\nGoodbye! 👋")
            break
        except Exception as e:
            print(f"Error: {str(e)}")

# Uncomment the line below to start interactive session
# interactive_chat_session()

print("💡 Uncomment the last line in this cell to start an interactive chat session!")

## 8. Performance and Statistics

Let's analyze the performance and statistics of our RAG system:

In [ ]:
import time
from collections import defaultdict

def performance_test():
    """Run performance tests on the RAG system."""

    print("⚡ Running Performance Tests...\n")

    # Test queries for each use case
    test_data = {
        "it_helpdesk": [
            "computer slow",
            "password reset",
            "printer issue",
            "check printer01 status",
            "VPN connection"
        ],
        "customer_support": [
            "track order",
            "return policy",
            "order ORD123456 status",
            "shipping cost",
            "payment methods"
        ],
        "hr_assistant": [
            "time off request",
            "benefits info",
            "employee EMP001 leave",
            "company holidays",
            "training courses"
        ]
    }

    results = defaultdict(list)

    for use_case, queries in test_data.items():
        print(f"Testing {use_case}...")

        chatbot = RAGChatbot(use_case, enable_functions=True)

        for query in queries:
            start_time = time.time()
            response = chatbot.chat(query)
            end_time = time.time()

            duration = end_time - start_time
            results[use_case].append({
                'query': query,
                'duration': duration,
                'method': response['method'],
                'success': response['success']
            })

    # Analyze results
    print("\n📊 Performance Analysis:")
    print("=" * 50)

    for use_case, case_results in results.items():
        durations = [r['duration'] for r in case_results]
        avg_duration = sum(durations) / len(durations)
        min_duration = min(durations)
        max_duration = max(durations)

        success_rate = sum(1 for r in case_results if r['success']) / len(case_results)

        rag_count = sum(1 for r in case_results if r['method'] == 'rag_retrieval')
        func_count = sum(1 for r in case_results if r['method'] == 'function_calling')

        print(f"\n{use_case.title()}:")
        print(f"  Average Response Time: {avg_duration:.2f}s")
        print(f"  Min/Max Response Time: {min_duration:.2f}s / {max_duration:.2f}s")
        print(f"  Success Rate: {success_rate:.1%}")
        print(f"  RAG Responses: {rag_count}, Function Calls: {func_count}")

performance_test()

## 9. Summary and Next Steps

This notebook has demonstrated the complete RAG chatbot system with:

### ✅ Completed Features:
- **FAISS Vector Store**: Fast document similarity search
- **RAG Pipeline**: Langchain-based retrieval and generation
- **Function Calling**: Azure OpenAI dynamic function execution
- **Multiple Use Cases**: IT Helpdesk, Customer Support, HR Assistant
- **Mock Data**: Comprehensive datasets for testing

### 🚀 Usage Options:
1. **Command Line**: `python chatbot_main.py --use-case it_helpdesk`
2. **Web Interface**: `streamlit run streamlit_app.py`
3. **Jupyter Notebook**: This interactive demo

### 🔧 Customization:
- Add new use cases in `mock_data/`
- Extend functions in `rag_system/function_calling.py`
- Modify prompts in `rag_system/retrieval_chain.py`
- Adjust vector search parameters in `rag_system/vector_store.py`

### 📈 Performance Considerations:
- Vector search is typically fast (< 1s)
- Function calls add latency but provide real-time data
- RAG retrieval provides contextually accurate responses
- Conversation history maintains context across interactions

In [ ]:
print("🎉 RAG Chatbot System Demo Completed!")
print("\n📝 Key Takeaways:")
print("• Vector search enables fast, relevant document retrieval")
print("• RAG combines retrieval with generation for accurate responses")
print("• Function calling provides dynamic, real-time data access")
print("• Multiple use cases show system versatility")
print("• Modular design allows easy customization and extension")

print("\n🔗 Next Steps:")
print("1. Configure your .env file with Azure OpenAI credentials")
print("2. Install requirements: pip install -r requirements.txt")
print("3. Run the web interface: streamlit run streamlit_app.py")
print("4. Customize for your specific use case")
print("5. Deploy to production environment")

print("\n✨ Happy coding!")